In [42]:
import pandas as pd
import numpy as np

In [43]:
col_name = ["ProblemId","C","C++","Go","Java","Javascript","Python","Python3"]
df = pd.read_csv('../../data/processed/success_score.csv')
df.rename(columns = {'Unnamed: 0':'problemId'}, inplace = True)
df

,problemId,C,C++,Go,Java,Javascript,Python,Python3
0,5b5c8cd7276e2200208fed62,0.708418,0.741621,0.674481,0.730613,0.674481,0.674481,0.763862
1,5b645f711b2cd40020b6c771,0.697211,0.780235,0.663106,0.698383,0.663106,0.890666,0.741500
2,5b649819c0b1b80020eb30e1,0.782912,0.729926,0.697609,0.702172,0.697609,0.697609,0.747527
3,5b64600d1b2cd40020b6c772,0.736975,0.736975,0.624211,0.701462,0.624211,0.624211,0.736975
4,5b6c3ec18cab100020e963b7,0.707936,0.753305,0.676529,0.738346,0.676529,0.676529,0.785069
...,...,...,...,...,...,...,...,...
199,63361b9a125f8a00222e06ef,0.660756,0.791391,0.660756,0.880797,0.660756,0.660756,0.660756
200,633620e3fe6e62002205e71e,0.669491,0.738728,0.669491,0.669491,0.669491,0.669491,0.669491
201,6336ad26ee62f10022c977c2,0.705785,0.747667,0.705785,0.705785,0.705785,0.705785,0.705785
202,6336b760ee62f10022c97c37,0.683354,0.759061,0.683354,0.683354,0.683354,0.683354,0.683354


In [44]:
import pickle
def save_model(model, model_name):
    with open('../../models/'+model_name+'.pkl', 'wb') as file:
        pickle.dump(model, file)

In [45]:
import pickle
def load_model(model_name):
    with open('../../models/'+model_name+'.pkl', 'rb') as file:
        model = pickle.load(file)
    return model

In [46]:
from sklearn.neighbors import KNeighborsClassifier as KNN
def train_knn(df, n_neighbors, algorithm, model_name):
    X = df.drop('problemId', axis=1)
    Y = df['problemId']
    knn = KNN(n_neighbors = n_neighbors+1, algorithm = algorithm) 
    knn.fit(X, Y)
    save_model(knn, model_name)

In [47]:
train_knn(df, 3, 'ball_tree', 'knn_ball_tree')

In [48]:
import numpy as np
def knn_recommend(problemId, n_neighbors,df, knn):
    Y = df['problemId']
    to_predict = np.array(df[df['problemId']==problemId].drop('problemId', axis=1)).reshape(1, -1)
    distances, indices = knn.kneighbors(to_predict, n_neighbors=n_neighbors+1)
    rec = []
    for i in indices[0]:
        if Y.iloc[i] != problemId:
            rec.append(Y.iloc[i])
    return rec

In [49]:
knn_recommend('6336b760ee62f10022c97c37', 3,df, load_model('knn_model_3_ball_tree'))

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


['6335f3dd125f8a00222e06eb',
 '6256b3db9f86540024c5653f',
 '5f6ad8a34bf84b0022982aff']

In [50]:
X = df.drop('problemId', axis=1)
Y = df['problemId']
knn = KNN(n_neighbors = 10, algorithm = 'ball_tree') 
knn.fit(X, Y)

KNeighborsClassifier(algorithm='ball_tree', n_neighbors=10)

In [51]:
# store similar problems in a dictionary
similar_problems = {}
for i in range(len(Y)):
    similar_problems[Y.iloc[i]] = knn_recommend(Y.iloc[i], 10,df, knn)

# save similar problems
with open('../../data/processed/similar_problems.pkl', 'wb') as file:
    pickle.dump(similar_problems, file)
    

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does no

In [52]:
with open('../../data/processed/similar_problems.pkl', 'rb') as file:
    similar_problems_pickle = pickle.load(file)

similar_problems_pickle['6335f3dd125f8a00222e06eb']

['5fbfa34b9296860022aa89df',
 '5f6ad8a34bf84b0022982aff',
 '6336b760ee62f10022c97c37',
 '6055b8213d00c8002293e4ce',
 '604bab0ec244c800220fc49a',
 '633620e3fe6e62002205e71e',
 '6255468d5be5d3002424a735',
 '621a24954323e500241da25d',
 '61d95ad22d678f002403d02c',
 '5ca063dd08f90d00223749ea']

In [53]:
knn_recommend('6336b760ee62f10022c97c37', 10,df, load_model('knn_model_3_ball_tree'))

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


['6335f3dd125f8a00222e06eb',
 '6256b3db9f86540024c5653f',
 '5f6ad8a34bf84b0022982aff']